In [1]:
import pandas as pd
import os
from os.path import dirname

root_path = dirname(os.getcwd())

pd.set_option("display.max_columns", None)
data_dir = root_path + "/data/datasets/original/"
data_dir_processed = root_path + "/data/datasets/processed/"
data_dir_graphs = root_path + "/data/datasets/graphs/"

print(root_path, data_dir, data_dir_processed, data_dir_graphs, sep="\n")

/home/sebdis/ProcessMining/HGNN/HGNN_NA
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/original/
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/processed/
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/graphs/


In [2]:
dataset = "BPI_Challenge_2012_W_Complete"

In [3]:
import pm4py

log = pm4py.read_xes(f"datasets/original/{dataset}.xes.gz")
log = pm4py.convert_to_dataframe(log)
log.to_csv(f"datasets/original/{dataset}.csv")

/home/sebdis/ProcessMining/HGNN/hgnn_env/lib/python3.10/site-packages/pm4py/util/dt_parsing/parser.py:77: UserWarning: ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11
  warnings.warn(


parsing log, completed traces ::   0%|          | 0/9658 [00:00<?, ?it/s]

In [4]:
filename = data_dir + dataset + ".csv"
raw_data = pd.read_csv(filename, index_col=False)
raw_data = raw_data.drop(columns="Unnamed: 0")
raw_data.head()

,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ
0,10912.0,COMPLETE,W_Afhandelen leads,2011-10-01 10:10:25.759000+00:00,2011-10-01 09:58:30.533000+00:00,173712,30000
1,11019.0,COMPLETE,W_Completeren aanvraag,2011-10-01 13:03:35.216000+00:00,2011-10-01 09:58:30.533000+00:00,173712,30000
2,11180.0,COMPLETE,W_Completeren aanvraag,2011-10-03 11:17:29.105000+00:00,2011-10-01 09:58:30.533000+00:00,173712,30000
3,11180.0,COMPLETE,W_Completeren aanvraag,2011-10-03 14:42:55.584000+00:00,2011-10-01 09:58:30.533000+00:00,173712,30000
4,10912.0,COMPLETE,W_Afhandelen leads,2011-10-01 10:16:49.843000+00:00,2011-10-01 09:45:37.274000+00:00,173706,18000


In [5]:
tab_all = raw_data.rename(
    columns={"case:concept:name": "CaseID", "concept:name": "Activity"}
)
tab_all.head()

,org:resource,lifecycle:transition,Activity,time:timestamp,case:REG_DATE,CaseID,case:AMOUNT_REQ
0,10912.0,COMPLETE,W_Afhandelen leads,2011-10-01 10:10:25.759000+00:00,2011-10-01 09:58:30.533000+00:00,173712,30000
1,11019.0,COMPLETE,W_Completeren aanvraag,2011-10-01 13:03:35.216000+00:00,2011-10-01 09:58:30.533000+00:00,173712,30000
2,11180.0,COMPLETE,W_Completeren aanvraag,2011-10-03 11:17:29.105000+00:00,2011-10-01 09:58:30.533000+00:00,173712,30000
3,11180.0,COMPLETE,W_Completeren aanvraag,2011-10-03 14:42:55.584000+00:00,2011-10-01 09:58:30.533000+00:00,173712,30000
4,10912.0,COMPLETE,W_Afhandelen leads,2011-10-01 10:16:49.843000+00:00,2011-10-01 09:45:37.274000+00:00,173706,18000


In [6]:
tab_all["Activity"].unique()

array(['W_Afhandelen leads', 'W_Completeren aanvraag',
       'W_Nabellen offertes', 'W_Valideren aanvraag',
       'W_Nabellen incomplete dossiers', 'W_Beoordelen fraude'],
      dtype=object)

In [7]:
tab_all["time:timestamp"] = [x.split(".")[0] for x in tab_all["time:timestamp"]]
tab_all["case:REG_DATE"] = [x.split(".")[0] for x in tab_all["case:REG_DATE"]]
tab_all["time:timestamp"] = tab_all["time:timestamp"].str.replace("-", "/")
tab_all["time:timestamp"] = tab_all["time:timestamp"].str.split("+", expand=True)[0]
tab_all["case:REG_DATE"] = tab_all["case:REG_DATE"].str.replace("-", "/")
tab_all["case:REG_DATE"] = tab_all["case:REG_DATE"].str.split("+", expand=True)[0]
tab_all.head()

,org:resource,lifecycle:transition,Activity,time:timestamp,case:REG_DATE,CaseID,case:AMOUNT_REQ
0,10912.0,COMPLETE,W_Afhandelen leads,2011/10/01 10:10:25,2011/10/01 09:58:30,173712,30000
1,11019.0,COMPLETE,W_Completeren aanvraag,2011/10/01 13:03:35,2011/10/01 09:58:30,173712,30000
2,11180.0,COMPLETE,W_Completeren aanvraag,2011/10/03 11:17:29,2011/10/01 09:58:30,173712,30000
3,11180.0,COMPLETE,W_Completeren aanvraag,2011/10/03 14:42:55,2011/10/01 09:58:30,173712,30000
4,10912.0,COMPLETE,W_Afhandelen leads,2011/10/01 10:16:49,2011/10/01 09:45:37,173706,18000


In [8]:
from utils import translate_time

tab_all['time:timestamp'] = tab_all['time:timestamp'].apply(translate_time)
tab_all['case:REG_DATE'] = tab_all['case:REG_DATE'].apply(translate_time)
tab_all.head()

,org:resource,lifecycle:transition,Activity,time:timestamp,case:REG_DATE,CaseID,case:AMOUNT_REQ
0,10912.0,COMPLETE,W_Afhandelen leads,1.317457e+09,1.317456e+09,173712,30000
1,11019.0,COMPLETE,W_Completeren aanvraag,1.317467e+09,1.317456e+09,173712,30000
2,11180.0,COMPLETE,W_Completeren aanvraag,1.317633e+09,1.317456e+09,173712,30000
3,11180.0,COMPLETE,W_Completeren aanvraag,1.317646e+09,1.317456e+09,173712,30000
4,10912.0,COMPLETE,W_Afhandelen leads,1.317457e+09,1.317455e+09,173706,18000


In [9]:
categorical_columns = ['org:resource', 'Activity']
real_value_columns = ["time:timestamp", 'case:AMOUNT_REQ', 'case:REG_DATE']

In [10]:
tab_all = tab_all.drop(columns=["lifecycle:transition"])

In [11]:
tab_all.isnull().values.any()

True

In [12]:
tab_all = tab_all.fillna({key: "NAN" for key in categorical_columns})
tab_all = tab_all.fillna({key: -1 for key in real_value_columns})

In [13]:
from math import log
import numpy as np

min_time = tab_all['time:timestamp'].min() if tab_all['time:timestamp'].min() < tab_all['case:REG_DATE'].min() else tab_all['case:REG_DATE'].min()

tab_all['time:timestamp'] -= min_time
tab_all['time:timestamp'] = [ log(x)  if x > 0 else 0. for x in tab_all['time:timestamp'].values]
# tab_all['time:timestamp'] = np.log(tab_all['time:timestamp'])

tab_all.head()

,org:resource,Activity,time:timestamp,case:REG_DATE,CaseID,case:AMOUNT_REQ
0,10912.0,W_Afhandelen leads,10.442930,1.317456e+09,173712,30000
1,11019.0,W_Completeren aanvraag,10.707527,1.317456e+09,173712,30000
2,11180.0,W_Completeren aanvraag,12.260206,1.317456e+09,173712,30000
3,11180.0,W_Completeren aanvraag,12.316947,1.317456e+09,173712,30000
4,10912.0,W_Afhandelen leads,10.454063,1.317455e+09,173706,18000


In [14]:
tab_all['case:REG_DATE'] -= min_time
tab_all['case:REG_DATE'] = [ log(x)  if x > 0 else 0. for x in tab_all['case:REG_DATE'].values]
# tab_all['time:timestamp'] = np.log(tab_all['time:timestamp'])

tab_all.head()

,org:resource,Activity,time:timestamp,case:REG_DATE,CaseID,case:AMOUNT_REQ
0,10912.0,W_Afhandelen leads,10.442930,10.421865,173712,30000
1,11019.0,W_Completeren aanvraag,10.707527,10.421865,173712,30000
2,11180.0,W_Completeren aanvraag,12.260206,10.421865,173712,30000
3,11180.0,W_Completeren aanvraag,12.316947,10.421865,173712,30000
4,10912.0,W_Afhandelen leads,10.454063,10.398580,173706,18000


In [15]:
tab_all['case:AMOUNT_REQ'] = [log(int(x)) if x > 0 else 0 for x in tab_all['case:AMOUNT_REQ'].values]

In [16]:
from utils import get_resource_role_map
roles_map = get_resource_role_map(tab_all)

/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/utils.py:218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['ac_rl'] = self.data.apply(associations, axis=1)


20.0 Analysing resource pool 
40.0 Analysing resource pool 
60.0 Analysing resource pool 
80.0 Analysing resource pool 
100.0 Analysing resource pool 
('ROLES \n'
 '\n'
 "([{'role': 'Role 1', 'quantity': 49, 'members': [10912.0, 11019.0, 11180.0, "
 "'NAN', 10982.0, 11002.0, 11049.0, 11122.0, 10913.0, 10889.0, 11121.0, "
 '10939.0, 11009.0, 11201.0, 11119.0, 10861.0, 11203.0, 11181.0, 11189.0, '
 '10899.0, 11000.0, 10863.0, 11169.0, 11179.0, 11001.0, 10228.0, 10909.0, '
 '10789.0, 10881.0, 10910.0, 10929.0, 10931.0, 11259.0, 10779.0, 10914.0, '
 '10933.0, 11079.0, 10932.0, 10935.0, 11254.0, 11003.0, 11269.0, 112.0, '
 "11299.0, 10124.0, 11309.0, 11300.0, 11302.0, 11319.0]}, {'role': 'Role 2', "
 "'quantity': 9, 'members': [10629.0, 11339.0, 10972.0, 10609.0, 10821.0, "
 "11289.0, 10125.0, 10809.0, 10138.0]}, {'role': 'Role 3', 'quantity': 2, "
 "'members': [11304.0, 10188.0]}], [{'role': 'Role 1', 'resource': 10912.0}, "
 "{'role': 'Role 1', 'resource': 11019.0}, {'role': 'Role 1', 're

In [17]:
roles_map = {
    x["resource"] : x["role"]
    for x in roles_map
}
roles_map

{10912.0: 'Role 1',
 11019.0: 'Role 1',
 11180.0: 'Role 1',
 'NAN': 'Role 1',
 10982.0: 'Role 1',
 11002.0: 'Role 1',
 11049.0: 'Role 1',
 11122.0: 'Role 1',
 10913.0: 'Role 1',
 10889.0: 'Role 1',
 11121.0: 'Role 1',
 10939.0: 'Role 1',
 11009.0: 'Role 1',
 11201.0: 'Role 1',
 11119.0: 'Role 1',
 10861.0: 'Role 1',
 11203.0: 'Role 1',
 11181.0: 'Role 1',
 11189.0: 'Role 1',
 10899.0: 'Role 1',
 11000.0: 'Role 1',
 10863.0: 'Role 1',
 11169.0: 'Role 1',
 11179.0: 'Role 1',
 11001.0: 'Role 1',
 10228.0: 'Role 1',
 10909.0: 'Role 1',
 10789.0: 'Role 1',
 10881.0: 'Role 1',
 10910.0: 'Role 1',
 10929.0: 'Role 1',
 10931.0: 'Role 1',
 11259.0: 'Role 1',
 10779.0: 'Role 1',
 10914.0: 'Role 1',
 10933.0: 'Role 1',
 11079.0: 'Role 1',
 10932.0: 'Role 1',
 10935.0: 'Role 1',
 11254.0: 'Role 1',
 11003.0: 'Role 1',
 11269.0: 'Role 1',
 112.0: 'Role 1',
 11299.0: 'Role 1',
 10124.0: 'Role 1',
 11309.0: 'Role 1',
 11300.0: 'Role 1',
 11302.0: 'Role 1',
 11319.0: 'Role 1',
 10629.0: 'Role 2',
 113

In [18]:
tab_all["org:resource:role"] = [roles_map[role] for role in tab_all["org:resource"].values]

In [19]:
categorical_columns.append("org:resource:role")

In [20]:
tab_all["org:resource"] = [str(x) for x in tab_all["org:resource"].values]

In [21]:
tab_all.head()

,org:resource,Activity,time:timestamp,case:REG_DATE,CaseID,case:AMOUNT_REQ,org:resource:role
0,10912.0,W_Afhandelen leads,10.442930,10.421865,173712,10.308953,Role 1
1,11019.0,W_Completeren aanvraag,10.707527,10.421865,173712,10.308953,Role 1
2,11180.0,W_Completeren aanvraag,12.260206,10.421865,173712,10.308953,Role 1
3,11180.0,W_Completeren aanvraag,12.316947,10.421865,173712,10.308953,Role 1
4,10912.0,W_Afhandelen leads,10.454063,10.398580,173706,9.798127,Role 1


In [22]:
tab_all.to_csv(data_dir_processed + f"{dataset}_processed_all.csv", index=False)

# Split the dataset

In [23]:
split_ratio = 2 / 3

first_act_tab = (
    tab_all.groupby("CaseID").first().sort_values("time:timestamp").reset_index()
)
first_act_tab = first_act_tab[
    ~first_act_tab.duplicated(subset=["CaseID", "Activity"], keep="first")
]
first_act_tab = first_act_tab.reset_index(drop=True)

list_train_valid_cases = list(
    first_act_tab[: int(split_ratio * len(first_act_tab))]["CaseID"].unique()
)

list_train_cases = list_train_valid_cases[: int(len(list_train_valid_cases) * 0.8)]
tab_train = tab_all[tab_all["CaseID"].isin(list_train_cases)].reset_index(drop=True)
tab_train.to_csv(data_dir_processed+ f"{dataset}_processed_train.csv", index = False)

list_valid_cases = list_train_valid_cases[int(len(list_train_valid_cases) * 0.8) :]
tab_valid = tab_all[tab_all["CaseID"].isin(list_valid_cases)].reset_index(drop=True)
tab_valid.to_csv(data_dir_processed+f"{dataset}_processed_valid.csv", index = False)

list_test_cases = list(
    first_act_tab[int(split_ratio * len(first_act_tab)) :]["CaseID"].unique()
)
tab_test = tab_all[tab_all["CaseID"].isin(list_test_cases)].reset_index(drop=True)
tab_test.to_csv(data_dir_processed+ f"{dataset}_processed_test.csv", index = False)

In [24]:
tab_all.columns

Index(['org:resource', 'Activity', 'time:timestamp', 'case:REG_DATE', 'CaseID',
       'case:AMOUNT_REQ', 'org:resource:role'],
      dtype='object')

In [ ]:
tab_train

### Prepare the graphs

In [25]:
from utils import get_case_ids, is_static, get_one_hot_encoder, get_one_hot_encodings

from torch import tensor, max, int64, float32
from torch_geometric.data import HeteroData
import numpy as np

In [26]:
def get_node_features(dataset: pd.DataFrame, trace: pd.DataFrame, cat_features, real_features) -> dict:
    columns_static = [c for c in trace if is_static(trace[c])]

    res = {}

    for key in trace:
        values = trace[key].values
        
        
        if key in cat_features:
            onehot_encoder = get_one_hot_encoder(dataset, key)
            if key not in columns_static:
                res[key] = tensor(
                    get_one_hot_encodings(onehot_encoder, values),
                    dtype=float32,
                    requires_grad=True
                )
            else:
                res[key] = tensor(
                    get_one_hot_encodings(onehot_encoder, np.array([values[0]])),
                    dtype=float32,
                    requires_grad=True
                )
        if key in real_features:
            if key not in columns_static:
                res[key] = tensor(values,  dtype=float32,requires_grad=True)
            else:
                res[key] = tensor([values[0]], dtype=float32,requires_grad=True)
            res[key] = res[key].reshape(res[key].shape[0], 1)
        
    

    return res


In [27]:


def compute_edges_indexs(node_features: dict, prefix_len):
    res = {}
    keys = node_features.keys()
    # indexes = [[i, j] for i in range(prefix_len) for j in range(i + 1, prefix_len)]
    indexes = [[i, i + 1] for i in range(prefix_len-1)]
    # activities indexes
    for k in keys:
        if len(node_features[k]) != 1:
            if k == "Activity":
                res[(k, "followed_by", k)] = indexes
                for k2 in keys:
                    if k2 != k:
                        if len(node_features[k2]) == 1:
                            res[(k, "related_to", k2)] = [
                                [i, 0] for i in range(prefix_len)
                            ]
                        else:
                            res[(k, "related_to", k2)] = [
                                [i, i] for i in range(prefix_len)
                            ]
            else:
                res[(k, "related_to", k)] = indexes

    return res

In [28]:
import torch

from torch import cat

def compute_edges_features(node_features, edges_indexes, cat_features, real_features):
    res = {}

    for k in edges_indexes:
        if k[0] == k[2]:
            indexes = edges_indexes[k]
            res[k] = []
            
            if k[0] in cat_features:
                for i in indexes:
                        res[k].append(      
                            cat(
                                (
                                    tensor([torch.equal(node_features[k[0]][i[0]],node_features[k[0]][i[1]])], dtype=torch.float32),
                                    tensor([i[1] - i[0]], dtype=torch.float32),
                                )
                            )
                        )
            elif k[0] in real_features:
                for i in indexes:
                        res[k].append(
                            tensor(
                                [
                                    node_features[k[0]][i[1]]
                                    - node_features[k[0]][i[0]],
                                    i[1] - i[0],
                                ]
                            )
                        )
            

    return res

In [29]:
# from pprint import pprint as print

from torch import stack

def build_prefixes_graph_from_trace(dataset, trace, cat_features, real_features):
    X = []  # graphs
   

    node_features = get_node_features(dataset, trace, cat_features, real_features)
    
    
    prefix_lenghts = range(2, len(trace))
    # print(prefix_lenghts)
    for prefix in prefix_lenghts:
        # print(prefix)

        # init node types and features
        G = HeteroData()
        for k in node_features:
            G[k].x = node_features[k][:prefix]
        

        edges_indexes = compute_edges_indexs(node_features, prefix)

        edge_features = compute_edges_features(node_features, edges_indexes, cat_features, real_features)
        
        for k in edge_features:
            G[k].edge_attr = stack(edge_features[k])
            


        for k in edges_indexes:
            ce = [[], []]
            for i in range(len(edges_indexes[k])):
                ce[0].append(edges_indexes[k][i][0])
                ce[1].append(edges_indexes[k][i][1])
            edges_indexes[k] = ce

        for k in edges_indexes:
            G[k].edge_index = tensor(edges_indexes[k], dtype=int64)

        G.y = {}
        for k in node_features:
            if k in cat_features:
                G.y[k] = torch.max(node_features[k][0],0)[1] if len(node_features[k]) == 1 else torch.max(node_features[k][prefix], 0)[1]
            else:
                G.y[k] = torch.tensor([node_features[k][prefix][0]]) if len(node_features[k]) > 1 else torch.tensor([node_features[k][0][0]])
        
        X.append(G)

        
    return X

## Create the datasets

In [30]:
case_train_ids = get_case_ids(tab_train)
case_valid_ids = get_case_ids(tab_valid)
case_test_ids = get_case_ids(tab_test)

In [31]:
print(len(case_train_ids))
print(len(case_valid_ids))
print(len(case_test_ids))

5150
1288
3220


In [32]:
trace = (
        tab_train.query(f"CaseID == {case_train_ids[0]}")
        .reset_index()
        .drop(columns="index")
        .drop(columns="CaseID")
    )
trace 

,org:resource,Activity,time:timestamp,case:REG_DATE,case:AMOUNT_REQ,org:resource:role
0,10912.0,W_Afhandelen leads,10.442930,10.421865,10.308953,Role 1
1,11019.0,W_Completeren aanvraag,10.707527,10.421865,10.308953,Role 1
2,11180.0,W_Completeren aanvraag,12.260206,10.421865,10.308953,Role 1
3,11180.0,W_Completeren aanvraag,12.316947,10.421865,10.308953,Role 1


In [33]:
graphs = build_prefixes_graph_from_trace(tab_all, trace, categorical_columns, real_value_columns)

In [92]:
graphs[1].y

{'org:resource': tensor(42),
 'Activity': tensor(2),
 'time:timestamp': tensor([12.3169]),
 'case:REG_DATE': tensor([10.4219]),
 'case:AMOUNT_REQ': tensor([10.3090]),
 'org:resource:role': tensor(0)}

In [34]:
graphs[1].x_dict

{'org:resource': tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]], grad_fn=<SliceBackward0>),
 'Activity': tensor([[1., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0

In [ ]:
graphs[1].edge_items()

In [35]:
from tqdm.notebook import tqdm


print("Preparing training dataset...")

X_train = []


for i in tqdm(range(len(case_train_ids))):
    trace = (
        tab_train.query(f"CaseID == {case_train_ids[i]}")
        .reset_index()
        .drop(columns="index")
        .drop(columns="CaseID")
    )
    graphs = build_prefixes_graph_from_trace(dataset=tab_all, trace=trace, cat_features=categorical_columns, real_features=real_value_columns)
    
    # print(trace)
    # print([x.x_dict for x in graphs[:2]])
    # print(labels[:2])
    # break

    for i in range(len(graphs)):
        X_train.append(graphs[i])
        

# Y_train = tensor(Y_train, dtype=float32)

print("Done!\n\n")

Preparing training dataset...


  0%|          | 0/5150 [00:00<?, ?it/s]

Done!




In [36]:
print("Preparing validation dataset...")

X_valid = []


for i in tqdm(range(len(case_valid_ids))):
    trace = (
        tab_valid.query(f"CaseID == {case_valid_ids[i]}")
        .reset_index()
        .drop(columns="index")
        .drop(columns="CaseID")
    )
    graphs = build_prefixes_graph_from_trace(dataset=tab_all, trace=trace,cat_features=categorical_columns, real_features=real_value_columns)
    for i in range(len(graphs)):
        X_valid.append(graphs[i])
       

# Y_valid = tensor(Y_valid)

print("Done!\n\n")

Preparing validation dataset...


  0%|          | 0/1288 [00:00<?, ?it/s]

Done!




In [37]:
print("Preparing test dataset...")

X_test = []


for i in tqdm(range(len(case_test_ids))):
    trace = (
        tab_test.query(f"CaseID == {case_test_ids[i]}")
        .reset_index()
        .drop(columns="index")
        .drop(columns="CaseID")
    )
    graphs = build_prefixes_graph_from_trace(dataset=tab_all, trace=trace,cat_features=categorical_columns, real_features=real_value_columns)
    for i in range(len(graphs)):
        X_test.append(graphs[i])
     

# Y_test = tensor(Y_test)

print("Done!\n\n")

Preparing test dataset...


  0%|          | 0/3220 [00:00<?, ?it/s]

Done!




In [1]:
for x in X_train:
    print(x.x_dict)

NameError: name 'X_train' is not defined

### Let's Save the graph datasets

In [38]:
import pickle

with open(data_dir_graphs + dataset + "_TRAIN_event_prediction_FINAL.pkl", "wb") as f:
    pickle.dump(X_train, f)
with open(data_dir_graphs + dataset + "_VALID_event_prediction_FINAL.pkl", "wb") as f:
    pickle.dump(X_valid, f)
with open(data_dir_graphs + dataset + "_TEST_event_prediction_FINAL.pkl", "wb") as f:
    pickle.dump(X_test, f)